In [96]:
import pandas as pd
import numpy as np
from surprise import Reader
from surprise import Dataset

In [97]:
df = pd.read_csv("topical_chat.csv")
df

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


In [98]:
#Assign Numeric Values of 1 or 0 to Categorical Data: Happy, Neutral, Sad, Surprised, Curious to dive deeper (shortened to curious)
df['Curious'] = np.where(df['sentiment'].str.contains("Curious"), 1, 0)
df['Happy'] = np.where(df['sentiment'].str.contains("Happy"), 1, 0)
df['Sad'] = np.where(df['sentiment'].str.contains("Sad"), 1, 0)
df['Neutral'] = np.where(df['sentiment'].str.contains("Neutral"), 1, 0)
df['Surprised'] = np.where(df['sentiment'].str.contains("Surprised"), 1, 0)
df

,conversation_id,message,sentiment,Curious,Happy,Sad,Neutral,Surprised
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper,1,0,0,0,0
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper,1,0,0,0,0
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper,1,0,0,0,0
3,1,Google provides online related services and p...,Curious to dive deeper,1,0,0,0,0
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper,1,0,0,0,0
...,...,...,...,...,...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised,0,0,0,0,1
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper,1,0,0,0,0
188375,8628,I don't think I have either. That's an insane...,Curious to dive deeper,1,0,0,0,0
188376,8628,"I did, my little brother used to love Thomas ...",Happy,0,1,0,0,0


In [99]:
#Number of messages in each conversation
df1 = df['conversation_id'].value_counts()
#Copy datafram to manipulate
manipulated = df.copy()


In [257]:
#labels each conversation with two distinct users
manipulated["User"] = ""
current_user = 1
for x in range(df1[1]):
    if (x % 2 == 0):
        manipulated['User'][x] = current_user
    else:
        manipulated['User'][x] = current_user+1
current_user = current_user + 2
for i in range(len(df.index)-1):
    if (df['conversation_id'][i] == df['conversation_id'][i+1]):
        pass
    else:
        for y in range(df1[df['conversation_id'][i+1]]):
            if (y % 2 == 0):
                manipulated['User'][i+1+y] = current_user
            else:
                manipulated['User'][i+1+y] = current_user+1
        current_user = current_user + 2

<ipython-input-257-4a1339a11532>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][x] = current_user
<ipython-input-257-4a1339a11532>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][x] = current_user+1
<ipython-input-257-4a1339a11532>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manipulated['User'][i+1+y] = current_user
<ipython-input-257-4a1339a11532>:18: SettingWithCopyWarning: 
A value is

In [117]:
data = [manipulated["Curious"], manipulated["Happy"], manipulated["Sad"], manipulated["Neutral"], manipulated["Surprised"]]

headers = ["Curious", "Happy", "Sad", "Neutral", "Surprised"]

feature_target = pd.concat(data, axis=1, keys=headers)
feature_target[0:10]

,Curious,Happy,Sad,Neutral,Surprised
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,1,0,0,0,0
9,1,0,0,0,0


In [115]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T



In [118]:
dfsample = feature_target[0:25]
df5 = pd.DataFrame([[, 6], [7, 8]], columns=list('AB'))
df.append(df2)

R = np.array(dfsample)
N = len(R)
M = len(R[0])
K = 5
P = np.random.rand(N,K)

Q = np.random.rand(M,K)
nP, nQ = matrix_factorization(R, P, Q, K)
nR = np.dot(nP, nQ.T)


In [119]:
nR

array([[0.97834308, 1.36825675, 1.47215662, 1.10754455, 1.33114508],
       [1.02931511, 1.55169385, 1.94046549, 1.37723201, 1.38315242],
       [0.96902593, 1.20318032, 1.3217868 , 0.67636919, 1.25973759],
       [1.068211  , 1.60219325, 2.05516559, 1.41413516, 1.30572249],
       [1.0417813 , 1.66349656, 1.55408767, 1.17690028, 1.40509597],
       [1.00856727, 1.70430482, 1.67040289, 1.38588984, 1.48368384],
       [0.9927557 , 1.49214321, 1.64839853, 1.38648902, 1.56865371],
       [1.00104883, 1.46164054, 1.84966289, 1.06223388, 1.31635465],
       [0.95384493, 1.67419003, 1.79145789, 1.51933003, 1.1583678 ],
       [1.03271174, 1.52872199, 1.23808746, 1.24641299, 1.46931577],
       [0.95755784, 1.31806072, 1.36624628, 0.75779414, 1.46268469],
       [0.9797118 , 1.43669348, 1.24184468, 1.13093271, 0.89529479],
       [1.03050332, 1.36897784, 1.15813816, 0.88189302, 1.41774676],
       [0.87187333, 0.99477445, 0.60975087, 0.60911237, 1.15211391],
       [0.95740121, 1.32566444, 0.

In [200]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('brown')

def analyze_blob_sentiment(text):
    blob = TextBlob(str(text))
    return blob.sentiment.polarity

def analyze_blob_subjectivity(text):
    blob = TextBlob(str(text))
    return blob.sentiment.subjectivity

def analyze_vader_sentiment(text):
    vader_analyzer = SentimentIntensityAnalyzer()
    sentiment_dict = vader_analyzer.polarity_scores(text)
    return sentiment_dict['compound']

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mnaga\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mnaga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\mnaga\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [201]:
analyze_vader_sentiment("Are you a fan of Google or Microsoft?")

0.3182

In [258]:
manipulated = manipulated.sample(frac=1).reset_index(drop=True)
split = np.array_split(manipulated, 100)

split1=split[0].copy()

split1

,conversation_id,message,sentiment,Curious,Happy,Sad,Neutral,Surprised,User,Sentiment Score,Vader Sentiment Score,Blob Subjectivity
0,4750,I loved Carol Burnett she was my favorite cel...,Curious to dive deeper,1,0,0,0,0,14432,0.0,0.6808,0.4625
1,5149,i agree. did you know the site has 58 total g...,Neutral,0,0,0,1,0,636,0.0,0.6808,0.4625
2,4129,Did you ever watch house?,Curious to dive deeper,1,0,0,0,0,8524,0.0,0.6808,0.4625
3,8233,I do follow football. Do you?,Happy,0,1,0,0,0,439,0.0,0.6808,0.4625
4,2021,"Same here, its been a great opportunity.",Curious to dive deeper,1,0,0,0,0,12654,0.0,0.6808,0.4625
...,...,...,...,...,...,...,...,...,...,...,...,...
1879,5213,Oh cool. I know many who have iphones. I am a...,Curious to dive deeper,1,0,0,0,0,13037,0.0,0.6808,0.4625
1880,252,"Yes, I do like horses, but I have never ridde...",Neutral,0,0,0,1,0,14500,0.0,0.6808,0.4625
1881,5344,I actually watch a lot of business and entrep...,Happy,0,1,0,0,0,3779,0.0,0.6808,0.4625
1882,171,"Yes, he is kind of sad. Well have a Merry Ch...",Neutral,0,0,0,1,0,9534,0.0,0.6808,0.4625


In [259]:
for i, row in split[0].iterrows():
    split1['Vader Sentiment Score'][i] = analyze_vader_sentiment(split[0]['message'].values[i])
    
    split1['Blob Subjectivity'][i] = analyze_blob_subjectivity(split[0]['message'].values[i])
split1


<ipython-input-259-0c9d89101e1a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split1['Vader Sentiment Score'][i] = analyze_vader_sentiment(split[0]['message'].values[i])
<ipython-input-259-0c9d89101e1a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split1['Blob Subjectivity'][i] = analyze_blob_subjectivity(split[0]['message'].values[i])


,conversation_id,message,sentiment,Curious,Happy,Sad,Neutral,Surprised,User,Sentiment Score,Vader Sentiment Score,Blob Subjectivity
0,4750,I loved Carol Burnett she was my favorite cel...,Curious to dive deeper,1,0,0,0,0,14432,0.0,0.9136,0.666667
1,5149,i agree. did you know the site has 58 total g...,Neutral,0,0,0,1,0,636,0.0,0.3612,0.625000
2,4129,Did you ever watch house?,Curious to dive deeper,1,0,0,0,0,8524,0.0,0.0000,0.000000
3,8233,I do follow football. Do you?,Happy,0,1,0,0,0,439,0.0,0.0000,0.000000
4,2021,"Same here, its been a great opportunity.",Curious to dive deeper,1,0,0,0,0,12654,0.0,0.7845,0.437500
...,...,...,...,...,...,...,...,...,...,...,...,...
1879,5213,Oh cool. I know many who have iphones. I am a...,Curious to dive deeper,1,0,0,0,0,13037,0.0,0.7988,0.683333
1880,252,"Yes, I do like horses, but I have never ridde...",Neutral,0,0,0,1,0,14500,0.0,0.8605,0.725000
1881,5344,I actually watch a lot of business and entrep...,Happy,0,1,0,0,0,3779,0.0,0.6808,0.358333
1882,171,"Yes, he is kind of sad. Well have a Merry Ch...",Neutral,0,0,0,1,0,9534,0.0,0.6003,0.950000


In [234]:
factorization_data = [split1["Vader Sentiment Score"], split1["Blob Subjectivity"]]

headers1 = ["Vader Sentiment Score", "Blob Subjectivity"]

feature_target1 = pd.concat(factorization_data, axis=1, keys=headers1)
feature_target1[0:30]


,Vader Sentiment Score,Blob Subjectivity
0,0.6124,0.600000
1,0.7650,0.000000
2,-0.3865,0.910000
3,0.3400,0.083333
4,0.2225,0.000000
5,0.4003,0.480000
6,0.8442,0.500000
7,0.4588,0.500000
8,0.5563,0.583333
9,-0.1689,0.541667


In [226]:
R2 = np.array(feature_target1)

N2 = len(feature_target1)
M2 = len(feature_target1.iloc[0])
K2 = 2

P2 = np.random.rand(N2,K2)
Q2 = np.random.rand(M2,K2)

nP2, nQ2 = matrix_factorization(R2, P2, Q2, K2)
nR2 = np.dot(nP2, nQ2.T)


In [260]:

feature_target1['User'] = split1['User']
feature_target1

,Vader Sentiment Score,Blob Subjectivity,User
0,0.6124,0.600000,14432
1,0.7650,0.000000,636
2,-0.3865,0.910000,8524
3,0.3400,0.083333,439
4,0.2225,0.000000,12654
...,...,...,...
1879,0.2494,0.575000,13037
1880,0.4019,0.000000,14500
1881,0.0000,0.000000,3779
1882,0.8860,0.455556,9534


In [261]:
X = feature_target1.iloc[:, :-1].values
y = feature_target1.iloc[:, 2].values
y=y.astype('int')
y

array([14432,   636,  8524, ...,  3779,  9534,   370])

In [280]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)
y_test

array([ 9754, 10811, 12203,   555, 14002,  3395,  4995, 10467,  4239,
       15265,  7054, 11775, 14943, 10780,  4558,  4911,  5363,  5621,
        5623, 11701,  8378, 16674, 13947, 10126,  6270, 13132,  2558,
        8876,   869,  6232, 13070,  8524, 14696,  3458, 14761, 10897,
       15754,  2743, 15107, 12327,  3162, 14984,  1732,  8786,  4670,
        2455, 16943, 15859,  1787,  9211,  1400,  6666, 10604, 10451,
        3818, 14724,  7555,  9466,  7071, 11194,  5511,  2382, 13736,
        8447,  2565, 10724,  6341,  5407, 15113,  7142,  1526,  6641,
        3541,  2858,  1616,  4713, 11675,  7846,  3608, 11440,  8247,
       15456,  7326,  9713,  2135,  7048, 14839,  7739,  6575, 12875,
        8098,  9691, 13335,  5940,   781, 14982, 14621,  9405, 17043,
        7290, 14589, 17140,  6352, 10414,  6155, 13938,  7284,  5699,
       13036,  9037, 16391, 12558, 14858,  7133,  9160,  8341,  1443,
       17009, 14403,  2922,  6295,  6416, 12105,  6183,  5260,  1906,
       11974,  7799,

In [290]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel = 'linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [291]:
from sklearn import metrics
print(y_pred)

[10734 10580 16301 10580 10580  3182 10008 10734  4675  4675 13603  4675
  3182 16980  4675  4675  2611 16173 13603 13603 10580 13603 13603 16980
  2611  4675 10580 10580 10580  5166 10734 13603   189  3182 13603 10734
 10580  6992 13603 16173  4675  2611 10580 16980 16301  4675 16980 16301
 13603 16173 16301 13603 12175 16980  4675 13603 10580  6992 10580 16173
  4675 16980 16301 10580 10734 16173  5166 10580  6992 10580 16173 13603
 10734 10008  4675 16301 10580  4675 16980 10580 13603 10580  4675 13603
 16980 10734 16301 16301 13603 13603 12175  5166 10008 16301 16980 16301
 16980 10580 16173 16980 16301 16980 10580 13603 10008  4675 16173 13603
 10580   189 16980 10580 16301 16980 10580 16980  4675 16173   189  4675
  1469 13603 13603 10734 10580 10580  6254 13603 16980 10008 10580  1469
   189  4675  4675 13603 10734  4675 16980  6992 10580 13603 10734 16173
 16980 16173  4675 10580  4675 16301 13603 13603 16980 16980 16301 16980
  4675  5166 13603 13603 10008   189 16173 16173  4

In [292]:
print(feature_target1[feature_target1['User']== 9754].index.values)
print(feature_target1[feature_target1['User']== 10734].index.values)

[801]
[  49 1125]


In [283]:
print(feature_target1['Vader Sentiment Score'][801])
print(feature_target1['Blob Subjectivity'][801])

-0.3384
0.3


In [295]:
print(feature_target1['Vader Sentiment Score'][1125])
print(feature_target1['Blob Subjectivity'][1125])

0.0
0.3092592592592593


In [287]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)
y_pred1 = classifier.predict(X_test)
y_pred1

array([ 3283,  4182,  1179,  1387,  3223,  1387,  3343,  3283,    27,
        1550,  5464,   566,   772,    56,  2031,  1211,   268,  1895,
        2442,  5377,   690,   889,  6096,  3592,   759,  2071,  1134,
        4727,  2405,  6432,  3973,  2068,  1339,  2222,  3058,  3283,
         182,  5561,  2068,  3569,  2808,  3491,   145,    56,  5115,
        7320,    56,   806,  1431,  2271,   636,  1508,  2359,    56,
         561,   108,  4727,   428,  2291,   875,  2531,    56,  1880,
        1990,  2146,  7036,  4470,   182,  7383,    39,  1811,  1508,
        3461,  1124,  1657,  5166,   145,  5563,  3839,  1134, 10126,
         618,  2136,  2109,   393,  3461,  1179,   753,  1508,  2068,
         176,  3779,   161,   696,   261,   636,    56,   182,   178,
        3839,   806,  3633,  1990,  1907,  1124,  2531,   131,  1907,
        4351,  1339,  1390,    39,  2776,  3839,   145,    56,   972,
         178,  1900,  1657,   924,  1508, 10648,  4310,  5663,  1990,
        2193,  2687,

In [288]:
print(feature_target1[feature_target1['User']== 3283].index.values)

[52]


In [289]:
print(feature_target1['Vader Sentiment Score'][52])
print(feature_target1['Blob Subjectivity'][52])

-0.34
0.25892857142857145


In [299]:
import joblib

# Export the model to a file
joblib.dump(classifier, 'model.joblib')

['model.joblib']

In [300]:
X_test[20]

array([0.5118    , 0.54545455])

In [435]:
unique_users = pd.DataFrame(feature_target1['User'].unique())
data1 = unique_users[0].tolist()


#unique_data = pd.concat(data1, axis=1, names=['Series name'])
#unique_data.columns =['Unique Users']
#len(unique_data)

unique_data['new'] = 0

data1[0:10]


[14432, 636, 8524, 439, 12654, 6406, 2452, 772, 304, 5565]

In [ ]:

messages = []
for b in range(unique_users.shape[0]):
    messages.append(manipulated[manipulated['User'] == unique_users[0][b]]['message'])
messages


testframe = pd.DataFrame({'User': pd.Series([], dtype='int'),
                   'b': pd.Series([], dtype='object')})
testframe['User'] = unique_data['Unique Users']
testframe['b'] = messages
testframe

In [437]:
messages_dict = dict(zip(data1, messages))